In [ ]:
# === Cell 1: Unified Environment & Project-Wide Setup ===
import os, json, math, datetime as dt
from datetime import datetime  # for human-readable prints
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional (map preview): keep if you use geemap
try:
    import geemap
    GEEMAP_AVAILABLE = True
except Exception:
    GEEMAP_AVAILABLE = False

plt.style.use('seaborn-v0_8-whitegrid')
os.makedirs('outputs', exist_ok=True)

import ee

# --- Earth Engine authentication and initialization ---
# You can override with: export EE_PROJECT_ID=your-project-id
EE_PROJECT_ID = os.environ.get('EE_PROJECT_ID', 'nasa-flood')  # <— 원하는 기본 프로젝트 ID
def _ee_init():
    try:
        ee.Initialize()
        return "Initialized with default credentials"
    except Exception:
        try:
            ee.Initialize(project=EE_PROJECT_ID)
            return f"Initialized with project='{EE_PROJECT_ID}'"
        except Exception:
            print("Authenticating with Earth Engine...")
            ee.Authenticate()
            ee.Initialize(project=EE_PROJECT_ID)
            return f"Authenticated & initialized with project='{EE_PROJECT_ID}'"

print(_ee_init())
print(f"Current time: {datetime.now().isoformat(timespec='seconds')}")

# ===== Project-wide constants =====
CFG = {
    # AOIs (EPSG:4326)
    # Mekong Delta (VN lower basin); box chosen for stability/reproducibility
    "AOI_DELTA": ee.Geometry.Rectangle([104.30,  8.50, 106.90, 10.90], geodesic=False),
    # Tonlé Sap (KH); "Mekong's heartbeat"
    "AOI_TONLESAP": ee.Geometry.Rectangle([103.30, 12.00, 105.20, 13.70], geodesic=False),

    # Analysis windows
    "YEARS": list(range(2015, 2025)),
    "FLOOD_MONTHS": (8, 9),     # Aug–Sep (wet-season peak)
    "DROUGHT_MONTHS": (3, 4),   # Mar–Apr (dry-season trough)

    # Thresholds (physical/empirical, fixed for comparability)
    "TH_VV_DB": -16.0,
    "TH_VH_DB": -22.0,

    # Landsat5 baseline window (pre-major-dam reference)
    "BASELINE_YEARS": [2005, 2006, 2007, 2008],

    # Event markers for plots
    "EVENTS": {
        "JINGHONG_FLOW_CUT": "2019-07-15",  # Jinghong flow cut (smoking gun)
        "XIAOWAN_ONLINE":    "2009-01-01",
        "NUOZHADU_ONLINE":   "2012-01-01"
    }
}

# ===== Utilities for Earth Engine =====
def _daterange_of_year_months(year:int, m1:int, m2:int):
    """Return ISO start and inclusive end-of-month last day for [m1..m2]."""
    start = dt.date(year, m1, 1)
    if m2 == 12:
        end = dt.date(year+1, 1, 1) - dt.timedelta(days=1)
    else:
        end = dt.date(year, m2+1, 1) - dt.timedelta(days=1)
    return start.isoformat(), end.isoformat()

def s1_min(aoi, start, end, pol):
    """Min-composite Sentinel-1 GRD over period to stabilize speckle."""
    return (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filterBounds(aoi)
            .filterDate(start, end)
            .filter(ee.Filter.eq('instrumentMode','IW'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol))
            .select(pol)
            .min()
            .clip(aoi))

def classify_water(img_min, pol, threshold_db):
    """Water = backscatter < threshold (in dB); returns self-masked binary."""
    return img_min.lt(threshold_db).selfMask()

def area_km2(mask_img, aoi, scale=30, band_name='constant'):
    """Compute km² of a self-masked image."""
    area = (mask_img.multiply(ee.Image.pixelArea())
            .reduceRegion(ee.Reducer.sum(), aoi, scale, maxPixels=1e12))
    # selfMask() produces band 'constant'
    return ee.Number(area.get(band_name)).divide(1e6)

def landsat5_c2_sr_mask_scale(img):
    """L5 C2 L2 scaling + cloud/shadow masking."""
    qa = img.select('QA_PIXEL')
    cloud  = 1 << 3
    shadow = 1 << 4
    mask = qa.bitwiseAnd(cloud).eq(0).And(qa.bitwiseAnd(shadow).eq(0))
    optical = img.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal = img.select('ST_B6').multiply(0.00341802).add(149.0)
    return (img.addBands(optical, None, True)
               .addBands(thermal, None, True)
               .updateMask(mask))

def landsat5_median(aoi, start_date, end_date, months=None):
    """Median composite with optional wrapped-month filter ((11,12),(1,4))."""
    col = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
           .filterBounds(aoi)
           .filterDate(start_date, end_date)
           .map(landsat5_c2_sr_mask_scale))
    if months:
        # e.g., months=((11,12),(1,4)) to handle dry-season wrap
        f = ee.Filter.Or(
            ee.Filter.calendarRange(int(months[0][0]), int(months[0][1]), 'month'),
            ee.Filter.calendarRange(int(months[1][0]), int(months[1][1]), 'month')
        )
        col = col.filter(f)
    return col.median().clip(aoi)

def mndwi(img):  # SR_B2 (Green), SR_B5 (SWIR)
    return img.normalizedDifference(['SR_B2','SR_B5']).rename('MNDWI')

def water_mask_from_mndwi(img, threshold=0.0):
    return img.gt(threshold).selfMask()

def chirps_sum_mm(aoi, start, end):
    """Return AOI-mean of CHIRPS precipitation sum (mm) over [start,end]."""
    col = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
           .filterBounds(aoi).filterDate(start, end).select('precipitation'))
    total = col.sum().reduceRegion(ee.Reducer.mean(), aoi, 5000, maxPixels=1e12)
    return ee.Number(total.get('precipitation'))

# (Optional) quick summary print
print(f"AOI_DELTA bounds: {CFG['AOI_DELTA'].bounds().getInfo()['coordinates'][0][0]} …")
print(f"AOI_TONLESAP bounds: {CFG['AOI_TONLESAP'].bounds().getInfo()['coordinates'][0][0]} …")
print("Setup complete ✅")


In [ ]:
# === Cell 2: Parameters, ancillary & helpers ===
# 분석 윈도우(건기): CFG["DROUGHT_MONTHS"] 사용 (기본: 3~4월)
DRY_M1, DRY_M2 = CFG["DROUGHT_MONTHS"]

# 모폴로지 스무딩(스펙클/점상 오탐 억제)
SMOOTH_RADIUS_M = 30   # 20~50m 범위 내 튜닝 가능

# 경사도 마스크(평지 필터) — 홍수/수면 오탐 억제
SLOPE_DEG_MAX = 5
dem = ee.Image('USGS/NASADEM_HGT')  # NASA 데이터 활용
slope_deg = ee.Terrain.slope(dem)
flat_mask = slope_deg.lte(SLOPE_DEG_MAX)

# 이벤트 표기(그래프 수직선)
EVENTS = {
    "JINGHONG_FLOW_CUT": pd.to_datetime(CFG["EVENTS"]["JINGHONG_FLOW_CUT"])
}

def morph_open(img, radius_m=SMOOTH_RADIUS_M):
    return (img.focal_min(radius_m, 'circle', 'meters')
               .focal_max(radius_m, 'circle', 'meters'))

def morph_close(img, radius_m=SMOOTH_RADIUS_M):
    return (img.focal_max(radius_m, 'circle', 'meters')
               .focal_min(radius_m, 'circle', 'meters'))

def refine_binary(mask_img):
    return morph_close(morph_open(mask_img))

def s1_vh_dry_area_km2(aoi, start_iso, end_iso, threshold_db=CFG["TH_VH_DB"], apply_flat=True):
    """건기 기간 VH 최소합으로 수역(물+식생 거울반사) 탐지 → 면적(km²)"""
    vh_min = s1_min(aoi, start_iso, end_iso, 'VH')
    water_raw = classify_water(vh_min, 'VH', threshold_db)
    water_smooth = refine_binary(water_raw)
    if apply_flat:
        water_smooth = water_smooth.updateMask(flat_mask)
    return float(area_km2(water_smooth, aoi, 30).getInfo() or 0.0)

def dry_period_iso(year:int, m1:int=DRY_M1, m2:int=DRY_M2):
    return _daterange_of_year_months(year, m1, m2)


In [ ]:
# === Cell 3: Baselines — Dry-season water (Landsat5) & CHIRPS climatology ===
def baseline_dry_area_km2(aoi):
    """2005–2008 동안 건기(11~4월) Landsat5 MNDWI>0 수역 면적 평균 (km²)"""
    # 11–12월 OR 1–4월 래핑 필터
    months_wrap = ((11,12),(1,4))
    l5_dry = landsat5_median(aoi, f"{min(CFG['BASELINE_YEARS'])}-01-01",
                                  f"{max(CFG['BASELINE_YEARS'])}-12-31",
                                  months=months_wrap)
    mask = water_mask_from_mndwi(mndwi(l5_dry), threshold=0.0)
    return float(area_km2(mask, aoi, 30).getInfo() or 0.0)

def chirps_climatology_mm(aoi, months=(DRY_M1, DRY_M2), base_years=range(1981, 2011)):
    """1981–2010 장기평균(건기 기간 누적 강수 mm)과 표준편차(mm)"""
    sums = []
    for y in base_years:
        start, end = _daterange_of_year_months(y, months[0], months[1])
        val = float(chirps_sum_mm(aoi, start, end).getInfo() or 0.0)
        sums.append(val)
    arr = np.array(sums, dtype=float)
    return float(arr.mean()), float(arr.std(ddof=1)), pd.DataFrame({"year": list(base_years), "mm": arr})

# AOI별 기준선 계산
BASELINES = {}
for name, aoi in [("Mekong_Delta", CFG["AOI_DELTA"]), ("Tonle_Sap", CFG["AOI_TONLESAP"])]:
    print(f"[Baseline] {name}: computing dry water (Landsat5) & CHIRPS climatology ...")
    dry_km2 = baseline_dry_area_km2(aoi)
    clim_mean_mm, clim_std_mm, clim_df = chirps_climatology_mm(aoi, (DRY_M1, DRY_M2))
    BASELINES[name] = {
        "dry_area_km2": dry_km2,
        "clim_mean_mm": clim_mean_mm,
        "clim_std_mm":  clim_std_mm,
        "clim_df":      clim_df
    }
    print(f"  • Dry baseline area: {dry_km2:,.0f} km²")
    print(f"  • CHIRPS climatology (1981–2010): {clim_mean_mm:,.1f} ± {clim_std_mm:,.1f} mm")


In [ ]:
# === Cell 4: Yearly drought indicators & rice yield loss estimation ===

# (⚠️중요) 수확량 손실 추정 파라미터 — 반드시 README/슬라이드에 “가정”으로 명시
# - RAIN_ELASTICITY_PER_100MM: 건기 누적강수 -100mm일 때 수확량 손실(%) (예: 0.12 → 12%)
# - WATERAREA_ELASTICITY_PER_10PCT: 건기 수면면적 기준선 대비 -10%일 때 수확량 손실(%) (예: 0.08 → 8%)
# - WEIGHT_RAIN, WEIGHT_WATER: 두 신호 가중치의 합은 1.0 권장
RAIN_ELASTICITY_PER_100MM   = 0.12
WATERAREA_ELAS_PER_10PCT    = 0.08
WEIGHT_RAIN, WEIGHT_WATER   = 0.6, 0.4

def estimate_yield_loss_pct(anom_mm, dry_deficit_pct):
    """음수 강수 이상(anom_mm<0)과 건기 수면부족율(%)로부터 수확량 손실(%) 추정"""
    rain_term  = max(0.0, -anom_mm) / 100.0 * RAIN_ELASTICITY_PER_100MM
    water_term = (max(0.0, dry_deficit_pct) / 10.0) * WATERAREA_ELAS_PER_10PCT
    return float(min(100.0, max(0.0, WEIGHT_RAIN*rain_term + WEIGHT_WATER*water_term)))

rows = []
for name, aoi in [("Mekong_Delta", CFG["AOI_DELTA"]), ("Tonle_Sap", CFG["AOI_TONLESAP"])]:
    base = BASELINES[name]
    for y in CFG["YEARS"]:
        start, end     = dry_period_iso(y)
        precip_mm      = float(chirps_sum_mm(aoi, start, end).getInfo() or 0.0)
        anom_mm        = precip_mm - base["clim_mean_mm"]
        anom_pct       = (anom_mm / base["clim_mean_mm"] * 100.0) if base["clim_mean_mm"]>0 else 0.0
        spi_like       = (anom_mm / base["clim_std_mm"]) if base["clim_std_mm"]>0 else np.nan  # 표준화 이상치
        dry_area_km2   = s1_vh_dry_area_km2(aoi, start, end, CFG["TH_VH_DB"], apply_flat=True)
        # 기준선(과거 건기) 대비 현재 건기 수면 부족율(+는 물 부족)
        dry_deficit_pct = (base["dry_area_km2"] - dry_area_km2) / base["dry_area_km2"] * 100.0 if base["dry_area_km2"]>0 else 0.0
        # 수확량 손실(%) 추정
        yield_loss_pct = estimate_yield_loss_pct(anom_mm, dry_deficit_pct)

        rows.append({
            "aoi": name, "year": y,
            "precip_mm": precip_mm,
            "anom_mm": anom_mm, "anom_pct": anom_pct, "spi_like": spi_like,
            "dry_area_km2": dry_area_km2, "dry_deficit_pct": dry_deficit_pct,
            "yield_loss_pct": yield_loss_pct
        })

df_drought_yearly = pd.DataFrame(rows).sort_values(["aoi","year"]).reset_index(drop=True)
display(df_drought_yearly.head(6))
out_csv = "outputs/drought_yearly_2015_2024.csv"
df_drought_yearly.to_csv(out_csv, index=False)
print("Saved ->", out_csv)

print("\n[Assumptions for yield-loss model]")
print(f"  RAIN_ELASTICITY_PER_100MM   = {RAIN_ELASTICITY_PER_100MM:.3f}")
print(f"  WATERAREA_ELAS_PER_10PCT    = {WATERAREA_ELAS_PER_10PCT:.3f}")
print(f"  WEIGHT_RAIN, WEIGHT_WATER   = {WEIGHT_RAIN:.2f}, {WEIGHT_WATER:.2f}")


In [ ]:
# === Cell 5: Yearly visualizations ===
from matplotlib.ticker import FuncFormatter

def plot_yearly_for_aoi(df, aoi_label):
    sub = df[df["aoi"]==aoi_label].copy()

    # 1) 강수 vs 기후평균
    fig, ax = plt.subplots(figsize=(12,5))
    ax.bar(sub["year"], sub["precip_mm"], label="Dry-season precip (mm)")
    clim = BASELINES[aoi_label]["clim_mean_mm"]
    ax.axhline(clim, color='red', linestyle='--', label=f"Climatology 81–10: {clim:.0f} mm")
    ax.set_title(f"{aoi_label} — Dry-season precipitation vs climatology")
    ax.set_xlabel("Year"); ax.set_ylabel("mm")
    ax.grid(True, axis='y'); ax.legend(); plt.tight_layout()
    plt.savefig(f"outputs/drought_{aoi_label}_precip_vs_clim.png", dpi=200)
    plt.show()

    # 2) 건기 수면 vs 기준선
    fig, ax = plt.subplots(figsize=(12,5))
    ax.plot(sub["year"], sub["dry_area_km2"], marker='o', label="Dry-season water area (km²)")
    base_dry = BASELINES[aoi_label]["dry_area_km2"]
    ax.axhline(base_dry, color='red', linestyle='--', label=f"Pre-dam dry baseline: {base_dry:,.0f} km²")
    ax.set_title(f"{aoi_label} — Dry-season surface-water extent")
    ax.set_xlabel("Year"); ax.set_ylabel("km²")
    ax.yaxis.set_major_formatter(FuncFormatter(lambda v, p: f"{int(v):,}"))
    ax.grid(True); ax.legend(); plt.tight_layout()
    plt.savefig(f"outputs/drought_{aoi_label}_water_vs_baseline.png", dpi=200)
    plt.show()

    # 3) 추정 수확량 손실(%) — 이벤트 라인(2019-07) 보조표기
    fig, ax = plt.subplots(figsize=(12,5))
    ax.plot(sub["year"], sub["yield_loss_pct"], marker='s', linestyle='-', label="Estimated rice yield loss (%)")
    ax.set_title(f"{aoi_label} — Estimated rice yield loss (dry-season)")
    ax.set_xlabel("Year"); ax.set_ylabel("%")
    ax.grid(True); ax.legend(); plt.tight_layout()
    plt.savefig(f"outputs/drought_{aoi_label}_yield_loss.png", dpi=200)
    plt.show()

plot_yearly_for_aoi(df_drought_yearly, "Mekong_Delta")
plot_yearly_for_aoi(df_drought_yearly, "Tonle_Sap")


In [ ]:
# === Cell 6: Monthly drill-down 2019–2020 (event reconstruction) ===
def month_range(start_str, end_str):
    s = pd.to_datetime(start_str).to_period("M")
    e = pd.to_datetime(end_str).to_period("M")
    return [p.to_timestamp() for p in pd.period_range(s, e, freq='M')]

def s1_vh_monthly_area(aoi, y:int, m:int):
    start, end = _daterange_of_year_months(y, m, m)
    return s1_vh_dry_area_km2(aoi, start, end, CFG["TH_VH_DB"], apply_flat=True)

def chirps_monthly_mm(aoi, y:int, m:int):
    start, end = _daterange_of_year_months(y, m, m)
    return float(chirps_sum_mm(aoi, start, end).getInfo() or 0.0)

def monthly_event_study(aoi, aoi_label, start="2019-01-01", end="2020-12-31"):
    months = month_range(start, end)
    recs = []
    for ts in months:
        y, m = ts.year, ts.month
        try:
            w_km2 = s1_vh_monthly_area(aoi, y, m)
        except Exception:
            w_km2 = np.nan
        try:
            p_mm  = chirps_monthly_mm(aoi, y, m)
        except Exception:
            p_mm = np.nan
        recs.append({"date": ts.date(), "year": y, "month": m,
                     "dry_water_km2": w_km2, "precip_mm": p_mm})
    df = pd.DataFrame(recs)
    out = f"outputs/monthly_drought_{aoi_label}_2019_2020.csv"
    df.to_csv(out, index=False)
    print("Saved ->", out)
    return df

def plot_monthly(df, aoi_label, fname_png):
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"])
    fig, ax1 = plt.subplots(figsize=(14,6))

    # 왼쪽 축: 수면면적(건기에도 월변동 확인)
    ax1.plot(df["date"], df["dry_water_km2"], marker='o', linestyle='-', label="Surface water (km²)")
    ax1.set_ylabel("Water area (km²)")
    ax1.yaxis.set_major_formatter(FuncFormatter(lambda v, p: f"{int(v):,}"))

    # 오른쪽 축: 월 강수(mm)
    ax2 = ax1.twinx()
    ax2.bar(df["date"], df["precip_mm"], width=20, alpha=0.35, label="Precip (mm)")
    ax2.set_ylabel("Precip (mm)")

    # 이벤트 수직선 표기
    for key, t in EVENTS.items():
        ax1.axvline(t, color='red', linestyle='--', linewidth=2)
        ax1.text(t, ax1.get_ylim()[1]*0.95, key, color='red', rotation=90, va='top')

    fig.suptitle(f"{aoi_label} — Monthly water & precip (2019–2020)")
    ax1.set_xlabel("Date"); ax1.grid(True, axis='y')
    # 범례 통합
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1+lines2, labels1+labels2, loc="upper right")
    plt.tight_layout()
    plt.savefig(fname_png, dpi=200)
    plt.show()
    print("Saved ->", fname_png)

df_m_delta = monthly_event_study(CFG["AOI_DELTA"], "Mekong_Delta", "2019-01-01", "2020-12-31")
plot_monthly(df_m_delta, "Mekong_Delta", "outputs/monthly_delta_2019_2020.png")

df_m_ts = monthly_event_study(CFG["AOI_TONLESAP"], "Tonle_Sap", "2019-01-01", "2020-12-31")
plot_monthly(df_m_ts, "Tonle_Sap", "outputs/monthly_tonlesap_2019_2020.png")


In [ ]:
# === Cell 7: Programmatic summary ===
def pct(v): return f"{v:.1f}%"

summary_lines = ["[Note06 Summary — Drought & Economic Impact]"]
for aoi in ["Mekong_Delta", "Tonle_Sap"]:
    sub = df_drought_yearly[df_drought_yearly["aoi"]==aoi]
    worst = sub.sort_values("yield_loss_pct", ascending=False).head(1).iloc[0]
    summary_lines += [
        f"• {aoi}",
        f"  - Dry-season precip climatology: {BASELINES[aoi]['clim_mean_mm']:.0f} ± {BASELINES[aoi]['clim_std_mm']:.0f} mm (1981–2010)",
        f"  - Pre-dam dry baseline water: {BASELINES[aoi]['dry_area_km2']:,.0f} km²",
        f"  - Worst estimated yield-loss year: {int(worst['year'])} → {worst['yield_loss_pct']:.1f}% "
          f"(anom: {worst['anom_mm']:.0f} mm, dry-deficit: {worst['dry_deficit_pct']:.1f}%)"
    ]

summary_lines += [
    "",
    "[Assumptions]",
    f"  • RAIN_ELASTICITY_PER_100MM   = {RAIN_ELASTICITY_PER_100MM:.3f}",
    f"  • WATERAREA_ELAS_PER_10PCT    = {WATERAREA_ELAS_PER_10PCT:.3f}",
    f"  • WEIGHT_RAIN, WEIGHT_WATER   = {WEIGHT_RAIN:.2f}, {WEIGHT_WATER:.2f}",
    "  → 위 계수는 문헌/현장 데이터로 교체 가능하도록 파라미터화했음(README에 가정 명시)."
]

txt = "\n".join(summary_lines)
print(txt)

with open("outputs/note06_summary.txt","w",encoding="utf-8") as f:
    f.write(txt)
print("Saved -> outputs/note06_summary.txt")
